<a href="https://colab.research.google.com/github/ninja-marduk/ml_precipitation_prediction/blob/main/models/base_models_STHyMOUNTAIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Entrenamiento de Modelos Baseline para Predicción Espaciotemporal de Precipitación Mensual STHyMOUNTAIN

Este notebook implementa modelos baseline para la predicción de precipitaciones usando datos espaciotemporales.

## 🔍 Implementación de Modelos Avanzados y Técnicas de Validación

Además de los modelos tabulares baseline, implementaremos:

1. **Modelos de Deep Learning** para capturar patrones espaciales y temporales:
   - Redes CNN para patrones espaciales
   - Redes ConvLSTM para patrones temporales
   - Redes ConvGRU para patrones temporales

El objetivo es proporcionar una evaluación completa de diferentes enfoques de modelado para la predicción de precipitación en regiones montañosas.

In [3]:
# ───────────────────────────────────── imports ─────────────────────────────────────
from pathlib import Path
import sys, os, gc, warnings, shutil
import numpy as np, pandas as pd, xarray as xr
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (Input, Conv2D, ConvLSTM2D, BatchNormalization,
                                     MaxPooling2D, Dropout, Flatten, Dense, Reshape)
from tensorflow.keras.layers import (Input, ConvLSTM2D, SimpleRNN, Conv2D, BatchNormalization,
                                     Flatten, Dense, Add, LayerNormalization, TimeDistributed)
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             mean_absolute_percentage_error, r2_score)
import matplotlib.pyplot as plt, seaborn as sns
from sklearn.preprocessing import StandardScaler
# Agrega la raíz del proyecto al sys.path
sys.path.append(str(Path().resolve().parent))

from lib.ConvGRU2D import ConvGRU2D


# ───────────────────────── 0. Entorno ─────────────────────────
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    !git clone -q https://github.com/ninja-marduk/ml_precipitation_prediction.git
    %cd ml_precipitation_prediction
    !pip install -q -r requirements.txt xarray netCDF4 seaborn scikit-learn geopandas imageio
    BASE_PATH = Path('/content/drive/MyDrive/ml_precipitation_prediction')
else:
    BASE_PATH = Path.cwd()
    for p in [BASE_PATH, *BASE_PATH.parents]:
        if (p/'.git').exists():
            BASE_PATH = p; break

print('BASE_PATH =', BASE_PATH)
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid'); sns.set_context('notebook')

import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import imageio.v2 as imageio
import cartopy.crs as ccrs

# ───────────────────────── PATHS & CONST ─────────────────────────
DATA_FILE = BASE_PATH/'data'/'output'/(
    'complete_dataset_with_features_with_clusters_elevation_windows_imfs_with_onehot_elevation_clean.nc')
OUT_ROOT  = BASE_PATH/'models'/'output'/'Spatial_CONVRNN'
OUT_ROOT.mkdir(parents=True, exist_ok=True)
SHAPE_DIR = BASE_PATH/'data'/'input'/'shapes'
DEPT_GDF = gpd.read_file(SHAPE_DIR/'MGN_Departamento.shp')

INPUT_WINDOW = 60; HORIZON = 3; EPOCHS = 50; BATCH = 4; LR = 1e-3
PATIENCE = 6

# ───────────────────────── FEATURE SETS ─────────────────────────
BASE_FEATS = ['year','month','month_sin','month_cos','doy_sin','doy_cos',
              'max_daily_precipitation','min_daily_precipitation','daily_precipitation_std',
              'elevation','slope','aspect']
ELEV_CLUSTER = ['elev_high','elev_med','elev_low']
KCE_FEATS = BASE_FEATS + ELEV_CLUSTER
PAFC_FEATS= KCE_FEATS + ['total_precipitation_lag1','total_precipitation_lag2','total_precipitation_lag12']

EXPERIMENTS = {
    'BASIC': BASE_FEATS,
    'KCE'  : KCE_FEATS,
    'PAFC' : PAFC_FEATS
}

# ───────────────────────── DATASET ─────────────────────────
ds = xr.open_dataset(DATA_FILE)
lat, lon = len(ds.latitude), len(ds.longitude)
print(f"Dataset → time={len(ds.time)}, lat={lat}, lon={lon}")

# ───────────────────────── HELPERS ─────────────────────────

def windowed_arrays(X:np.ndarray, y:np.ndarray):
    """Crea ventanas deslizantes 5‑D para X y 4‑D para y."""
    seq_X, seq_y = [], []
    T = len(X)
    for start in range(T-INPUT_WINDOW-HORIZON+1):
        end_w = start+INPUT_WINDOW; end_y=end_w+HORIZON
        Xw = X[start:end_w]; yw = y[end_w:end_y]
        if np.isnan(Xw).any() or np.isnan(yw).any():
            continue
        seq_X.append(Xw); seq_y.append(yw)
    return np.array(seq_X,dtype=np.float32), np.array(seq_y,dtype=np.float32)


def quick_plot(ax,data,cmap,title,vmin=None,vmax=None):
    mesh = ax.pcolormesh(ds.longitude,ds.latitude,data,cmap=cmap,shading='nearest',
                         vmin=vmin,vmax=vmax,transform=ccrs.PlateCarree())
    ax.coastlines(); ax.add_geometries(DEPT_GDF.geometry,ccrs.PlateCarree(),edgecolor='black',facecolor='none',linewidth=1)
    ax.gridlines(draw_labels=True,top_labels=False,right_labels=False)
    ax.set_title(title,fontsize=9); return mesh

# ───────────────────────── MODEL FACTORIES ─────────────────────────

def build_conv_lstm(n_feats:int):
    inp = Input(shape=(INPUT_WINDOW,lat,lon,n_feats),name='in')
    x   = ConvLSTM2D(64,(3,3),padding='same',return_sequences=True)(inp)
    x   = ConvLSTM2D(32,(3,3),padding='same')(x)
    x   = Flatten()(x)
    x   = Dense(HORIZON*lat*lon,activation='linear')(x)
    out = Reshape((HORIZON,lat,lon,1))(x)
    m   = Model(inp,out,name='ConvLSTM'); m.compile('adam','mse'); return m


def build_conv_gru(n_feats:int):
    inp = Input(shape=(INPUT_WINDOW,lat,lon,n_feats))
    x = ConvGRU2D(64,(3,3),padding='same',return_sequences=True)(inp)
    x = ConvGRU2D(32,(3,3),padding='same')(x)
    x = Flatten()(x)
    x = Dense(HORIZON*lat*lon)(x)
    out=Reshape((HORIZON,lat,lon,1))(x)
    m=Model(inp,out,name='ConvGRU'); m.compile('adam','mse'); return m


def build_conv_rnn(n_feats:int):
    inp = Input(shape=(INPUT_WINDOW,lat,lon,n_feats))
    x = SimpleRNN(128,activation='tanh')(Flatten()(inp))
    x = Dense(HORIZON*lat*lon)(x)
    out=Reshape((HORIZON,lat,lon,1))(x)
    m=Model(inp,out,name='ConvRNN'); m.compile('adam','mse'); return m

MODELS = {
    'ConvLSTM': build_conv_lstm,
    'ConvGRU' : build_conv_gru,
    'ConvRNN' : build_conv_rnn
}

# ───────────────────────── TRAIN + EVAL LOOP ─────────────────────────
results=[]
for exp, feat_list in EXPERIMENTS.items():
    print(f"\n=== Experimento {exp} ({len(feat_list)} features) ===")
    Xarr = ds[feat_list].to_array().transpose('time','latitude','longitude','variable').values.astype(np.float32)
    yarr = ds['total_precipitation'].values.astype(np.float32)[...,None]  # (T,H,W,1)

    X, y = windowed_arrays(Xarr, yarr)
    split=int(0.8*len(X))

    sx = StandardScaler().fit(X[:split].reshape(-1,len(feat_list)))
    sy = StandardScaler().fit(y[:split].reshape(-1,1))

    X_sc = sx.transform(X.reshape(-1,len(feat_list))).reshape(X.shape)
    y_sc = sy.transform(y.reshape(-1,1)).reshape(y.shape)

    X_tr, X_va = X_sc[:split], X_sc[split:]
    y_tr, y_va = y_sc[:split], y_sc[split:]

    OUT_EXP = OUT_ROOT/exp; OUT_EXP.mkdir(exist_ok=True)

    for mdl_name,builder in MODELS.items():
        print(f"→ {mdl_name}")
        model_path = OUT_EXP/f"{mdl_name.lower()}_best.keras"
        if model_path.exists():
            print("⏩ ya entrenado, cargando …"); model=tf.keras.models.load_model(model_path,compile=False)
        else:
            model = builder(n_feats=len(feat_list))
            cb = [EarlyStopping('val_loss',patience=PATIENCE,restore_best_weights=True),
                  ModelCheckpoint(model_path,save_best_only=True)]
            model.fit(X_tr,y_tr,validation_data=(X_va,y_va),epochs=EPOCHS,batch_size=BATCH,callbacks=cb,verbose=0)
        # --- Predicción último batch validación ---
        y_hat_sc = model.predict(X_va[-1:],verbose=0)
        y_hat    = sy.inverse_transform(y_hat_sc.reshape(-1,1)).reshape(HORIZON,lat,lon)
        y_true   = sy.inverse_transform(y_va[-1:].reshape(-1,1)).reshape(HORIZON,lat,lon)

        # --- Mapas & GIF ---
        vmin=0; vmax=max(y_true.max(),y_hat.max())
        frames=[]; dates=pd.date_range(ds.time.values[-HORIZON],periods=HORIZON,freq='MS')
        for h in range(HORIZON):
            err = np.clip(np.abs((y_true[h]-y_hat[h])/(y_true[h]+1e-5))*100,0,100)
            fig,axs=plt.subplots(1,3,figsize=(12,4),subplot_kw={'projection':ccrs.PlateCarree()})
            quick_plot(axs[0],y_true[h],'Blues',f"Real h={h+1}",vmin,vmax)
            quick_plot(axs[1],y_hat[h],'Blues',f"Pred {mdl_name} h={h+1}",vmin,vmax)
            quick_plot(axs[2],err,'Reds',f"MAPE% h={h+1}",0,100)
            fig.suptitle(f"{mdl_name} – {exp} – {dates[h].strftime('%Y-%m')}")
            png = OUT_EXP/f"{mdl_name}_{h+1}.png"; plt.savefig(png,bbox_inches='tight'); plt.close(fig)
            frames.append(imageio.imread(png))
        imageio.mimsave(OUT_EXP/f"{mdl_name}.gif",frames,fps=0.5)

        # --- Métricas ---
        for h in range(HORIZON):
            results.append({
                'Experiment':exp,'Model':mdl_name,'H':h+1,
                'RMSE':np.sqrt(mean_squared_error(y_true[h].ravel(),y_hat[h].ravel())),
                'MAE': mean_absolute_error(y_true[h].ravel(),y_hat[h].ravel()),
                'R2':  r2_score(y_true[h].ravel(),y_hat[h].ravel())
            })
        tf.keras.backend.clear_session(); gc.collect()

# ───────────────────────── CSV FINAL ─────────────────────────
res_df=pd.DataFrame(results)
res_df.to_csv(OUT_ROOT/'metrics_spatial.csv',index=False)
print("\n📑 Metrics saved →", OUT_ROOT/'metrics_spatial.csv')


BASE_PATH = /Users/riperez/Conda/anaconda3/envs/precipitation_prediction/github.com/ml_precipitation_prediction
Dataset → time=518, lat=61, lon=65

=== Experimento BASIC (12 features) ===
→ ConvLSTM


: 